# Summarizing the regression deconvolution effects for latter plotting

### What this does:
- Loads in the GT effects data
    - Including which compounds were "significant" GT hits (Mahalanobis distance > mean(Mahala. dist) + std(Mahala. dist))
- For all CS screens
    - Reads in the results of decon at many permutation testing stringencies
    - Saves the scaled L1 norm for each perturbation as a measure of CS effect
- For each CS screen at each stringency level of permutation testing
    - Calulcate Pearson & Spearman correlation between GT effects and CS inferred effects
    - Calculate TPR and FPR for finding significant hits from GT
        - True positive = Hit in CS is a significant hit in GT
        - False positive = Hit in CS is not aa significant hit in GT
        - True negative = Not a hit in CS and is not a significant hit in GT
        - False negative = Not a hit in CS and is a significant hit in GT

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scanpy as sc
import anndata

from scipy.stats import pearsonr,hypergeom,spearmanr

# Loading in the GT results

In [2]:
# Load in the set of 316 drugs used in the screens
cs_drugs = pd.read_csv("../screening_methods/mahalanobis_calcs_on_compressed_15Nov2021/mahala_calculations/CS_run1_2.0x5.0r_randommahala_table.csv").Metadata_perturbation.values
gt_mahala = pd.read_csv("../../../BEM/Manu_analysis/2_OG316_analysis/GT316_mahala_empirical_table.csv")

# Need to update the name of one drug with a weird name that got annotated in two ways
gt_mahala.Metadata_perturbation.values[gt_mahala.Metadata_perturbation.values=='Methyldopa Sesquihydrate (L-A-Methyl-Dopa Sesquihyrate)']='Methyldopa Sesquihydrate (L-_-Methyl-Dopa Sesquihyrate)'

# Filter down to OG 316 and remove the DMSO wells
gt_mahala = gt_mahala.loc[gt_mahala.Metadata_perturbation.isin(cs_drugs)]
gt_mahala = gt_mahala.loc[gt_mahala.Metadata_perturbation!="DMSO"]


# rename columns where necesseary
gt_mahala.columns = ['Metadata_perturbation','gt_mahalanobis','gt_signif_mahala']
gt_mahala.index = gt_mahala.Metadata_perturbation.values
gt_mahala.sort_index(inplace=True)

gt_mahala.gt_signif_mahala = 'False'
# gt_mahala.gt_signif_mahala.loc[gt_mahala.gt_mahalanobis > np.percentile(gt_metrics.gt_mahalanobis,95)] = 'True'
# gt_mahala.gt_signif_mahala.loc[gt_mahala.gt_mahalanobis > np.mean(gt_metrics.gt_mahalanobis) + 2*np.std(gt_metrics.gt_mahalanobis)] = 'True'
sig_mahala_drugs = pd.read_csv("GT_316_sig_mahala.csv",index_col=0)
gt_mahala.gt_signif_mahala.loc[np.isin(gt_mahala.Metadata_perturbation,sig_mahala_drugs.Metadata_perturbation)] = 'True'

# Loading in the GT phenotyping results
gt_leiden = pd.read_csv("GT316_drug_clusters.csv",index_col=0)
# Need to update the name of one drug with a weird name that got annotated in two ways
gt_leiden.Metadata_perturbation.values[gt_leiden.Metadata_perturbation.values=='Methyldopa Sesquihydrate (L-A-Methyl-Dopa Sesquihyrate)']='Methyldopa Sesquihydrate (L-_-Methyl-Dopa Sesquihyrate)'
gt_leiden['gt_sig_leiden'] = True
gt_leiden.columns.values[1] = 'gt_leiden'

# Combining mahalanobis and phenotyping results
gt_metrics = gt_mahala.merge(gt_leiden,how='left')
gt_metrics.index = gt_metrics.Metadata_perturbation

/var/folders/7v/c1jnbc_12kq312b56vlhgmv40000gn/T/ipykernel_5486/331540485.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gt_mahala.gt_signif_mahala.loc[np.isin(gt_mahala.Metadata_perturbation,sig_mahala_drugs.Metadata_perturbation)] = 'True'


# Loading in the CS regression deconvolution results and assesing effect sizes for each compound in each screen

In [6]:
data_read_path = "../../../cell_painting_data_lock/4_CS_OBJECTS_median_aggregated/"
data_read_suffix = "_PCH_rd_OG316"
save_path = "regression_summary_stats/"
sc.settings.figdir = save_path

regression_read_path = "../regression_deconvolution_December2021/decon_out/raw_rd_OG316_raw_decon_out/"

permute_thresholds = np.arange(0.0,1.01,step=0.01).round(2).astype(str)
for permute_threshold in permute_thresholds:
    print(permute_threshold)
    runs = ['CS_run1','CS_run2','CS_run3']
    for run in runs:
        print(run)
        metadata = pd.read_csv(data_read_path+run+data_read_suffix+"_metadata.csv",index_col=0)
         # Layout all of the compression schemes
        compression_methods = metadata.groupby(['Metadata_compression','Metadata_replicates','Metadata_perturbation','Metadata_Plate']).size().reset_index().rename(columns={0:''})
        compression_methods = compression_methods.loc[np.isin(compression_methods['Metadata_perturbation'],['random','random1','random2'])]
        compression_methods.to_csv(run+"_compression_methods.csv",index=False)
        for i in range(compression_methods.shape[0]):
            print(i)
            compression = compression_methods['Metadata_compression'].iloc[i]
            replicates = compression_methods['Metadata_replicates'].iloc[i]
            scheme = compression_methods['Metadata_perturbation'].iloc[i]
            save_prefix = str(compression) + "x"+str(replicates) + "r" +"_"+str(scheme)



            # Loading in the permuted & non-permuted regression results
            model_coef = pd.read_csv(regression_read_path +run+"_"+str(compression)+"x_"+str(replicates)+"r"+"_"+scheme+"_model_coef.csv",index_col=0)
            model_coef.index.rename('Metadata_perturbation',inplace=True)
            model_coef = model_coef.loc[model_coef.index.values!="DMSO"]
            model_coef.sort_index(inplace=True)
            model_coef['regression_l1_norm'] = np.abs(model_coef).sum(1)

            permute_model_coef = pd.read_csv(regression_read_path +run+"_"+str(compression)+"x_"+str(replicates)+"r"+"_"+scheme+"_permute_"+permute_threshold+"_model_coef.csv",index_col=0)
            permute_model_coef.index.rename('Metadata_perturbation',inplace=True)
            permute_model_coef = permute_model_coef.loc[permute_model_coef.index.values!="DMSO"]
            permute_model_coef.sort_index(inplace=True)
            permute_model_coef['regression_l1_norm'] = np.abs(permute_model_coef).sum(1)


            # For now removing the reg leidens
            permute_model_coef['permute_reg_l1_norm'] = permute_model_coef.regression_l1_norm
            permute_model_coef['permute_reg_l1_scaled'] = permute_model_coef.regression_l1_norm /np.max(permute_model_coef.regression_l1_norm)
            reg_metrics = pd.concat([model_coef.regression_l1_norm,
                      permute_model_coef.permute_reg_l1_norm,permute_model_coef.permute_reg_l1_scaled],
                      sort=True,axis=1)
            reg_metrics.Metadata_perturbation = reg_metrics.index.values

            cs_metrics = pd.concat([reg_metrics,gt_metrics],sort=True,axis=1)
            cs_metrics.index = np.arange(316)
            cs_metrics['Metadata_run'] = run
            cs_metrics['Metadata_compresion'] = compression
            cs_metrics['Metadata_replicates'] = replicates
            cs_metrics['Metadata_scheme'] = scheme
            new_order = cs_metrics.columns.values[9:13].tolist()+cs_metrics.columns.values[0:4].tolist()+cs_metrics.columns.values[5:9].tolist()
            # cs_metrics = cs_metrics[new_order]
            cs_metrics = cs_metrics.loc[:,~cs_metrics.columns.duplicated()]

            if i ==0 and run =="CS_run1":
                combined_metrics = cs_metrics
            else:
                combined_metrics = pd.concat([combined_metrics,cs_metrics],axis=0)
    combined_metrics.to_csv(save_path+"regression_metrics_permute_"+permute_threshold+"_Dec102021.csv")     

0.0
CS_run1
0


/var/folders/7v/c1jnbc_12kq312b56vlhgmv40000gn/T/ipykernel_5486/245974346.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reg_metrics.Metadata_perturbation = reg_metrics.index.values


1


/var/folders/7v/c1jnbc_12kq312b56vlhgmv40000gn/T/ipykernel_5486/245974346.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reg_metrics.Metadata_perturbation = reg_metrics.index.values


2


...

/var/folders/7v/c1jnbc_12kq312b56vlhgmv40000gn/T/ipykernel_5486/245974346.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reg_metrics.Metadata_perturbation = reg_metrics.index.values
/var/folders/7v/c1jnbc_12kq312b56vlhgmv40000gn/T/ipykernel_5486/245974346.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reg_metrics.Metadata_perturbation = reg_metrics.index.values


5
6


/var/folders/7v/c1jnbc_12kq312b56vlhgmv40000gn/T/ipykernel_5486/245974346.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reg_metrics.Metadata_perturbation = reg_metrics.index.values
/var/folders/7v/c1jnbc_12kq312b56vlhgmv40000gn/T/ipykernel_5486/245974346.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reg_metrics.Metadata_perturbation = reg_metrics.index.values


7


/var/folders/7v/c1jnbc_12kq312b56vlhgmv40000gn/T/ipykernel_5486/245974346.py:48: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reg_metrics.Metadata_perturbation = reg_metrics.index.values


# Calculate summary statistics over all compounds for each screen

In [35]:
data_read_path = "../../../cell_painting_data_lock/4_CS_OBJECTS_median_aggregated/"
data_read_suffix = "_PCH_rd_OG316"
save_path = "regression_summary_stats/"
permute_thresholds = np.arange(0.0,1.01,step=0.01).round(2).astype(str)
for permute_threshold in permute_thresholds:
    combined_metrics = pd.read_csv(save_path+"regression_metrics_permute_"+permute_threshold+"_Dec102021.csv",index_col=0)

    run_vals = []
    compression_vals = []
    rep_vals = []
    scheme_vals = []

    pearson_correlations = []
    spearman_correlations = []

    tpr = []
    fpr = []
    tp = []
    fp = []
    tn = []
    fn = []
    precision = []
    recall = []


    for run in ['CS_run1','CS_run2','CS_run3']:
        print(run)
        metadata = pd.read_csv(data_read_path+run+data_read_suffix+"_metadata.csv",index_col=0)
         # Layout all of the compression schemes
        compression_methods = metadata.groupby(['Metadata_compression','Metadata_replicates','Metadata_perturbation','Metadata_Plate']).size().reset_index().rename(columns={0:''})
        compression_methods = compression_methods.loc[np.isin(compression_methods['Metadata_perturbation'],['random','random1','random2'])]
        compression_methods.to_csv(run+"_compression_methods.csv",index=False)

        for i in range(compression_methods.shape[0]):
        
            compression = compression_methods['Metadata_compression'].iloc[i]
            replicates = compression_methods['Metadata_replicates'].iloc[i]
            scheme = compression_methods['Metadata_perturbation'].iloc[i]
            print(str(compression)+"_"+str(replicates)+scheme)
            # Finding the drugs with at least one non zero coeficient
            cs_metrics = combined_metrics.loc[(combined_metrics.Metadata_run==run)&
                          (combined_metrics.Metadata_compresion==compression)&
                          (combined_metrics.Metadata_replicates==replicates)&
                                (combined_metrics.Metadata_scheme==scheme)&
                               (combined_metrics.permute_reg_l1_norm>0)]
            
            cs_metrics_all_perturbs = combined_metrics.loc[(combined_metrics.Metadata_run==run)&
              (combined_metrics.Metadata_compresion==compression)&
              (combined_metrics.Metadata_replicates==replicates)&
                    (combined_metrics.Metadata_scheme==scheme)]
            top_hit_threshold = np.mean(cs_metrics_all_perturbs.permute_reg_l1_norm) + 2*np.std(cs_metrics_all_perturbs.permute_reg_l1_norm)
            top_cs_all = cs_metrics_all_perturbs.loc[cs_metrics_all_perturbs.permute_reg_l1_norm > np.mean(cs_metrics_all_perturbs.permute_reg_l1_norm) + 2*np.std(cs_metrics_all_perturbs.permute_reg_l1_norm)]
     
            #Correlations
            pearson_correlations.append(pearsonr(cs_metrics.permute_reg_l1_scaled,cs_metrics.gt_mahalanobis)[0])
            spearman_correlations.append(spearmanr(cs_metrics.permute_reg_l1_scaled,cs_metrics.gt_mahalanobis)[0])

            run_vals.append(run)
            compression_vals.append(compression)
            rep_vals.append(replicates)
            scheme_vals.append(scheme)

            # TPR, FPR, Precision, and Recall calculations

            total_positives = cs_metrics.shape[0]
            total_negatives = gt_metrics.shape[0] -total_positives

            true_positives = sum(cs_metrics.gt_signif_mahala==True)
            tp.append(true_positives)
            false_positives = total_positives - true_positives
            fp.append(false_positives)

            negative_data = gt_metrics.loc[np.isin(gt_metrics.Metadata_perturbation,cs_metrics.Metadata_perturbation,invert=True)]
            false_negatives = sum(negative_data.gt_signif_mahala=='True')
            fn.append(false_negatives)
            true_negatives = sum(negative_data.gt_signif_mahala!='True')
            tn.append(true_negatives)

            tpr.append(true_positives/(true_positives + false_negatives))

            fpr.append(false_positives/(false_positives + true_negatives))

            if permute_threshold == "0.0":
                 precision.append(-1)
                 recall.append(-1)
            else:
                precision.append(true_positives/(true_positives+false_positives))
                recall.append(true_positives/(true_positives+false_negatives))

    regression_hit_nomination_stats = pd.DataFrame({"Metadata_run":run_vals,
                                                   "Metadata_compression":compression_vals,
                                                   "Metadata_replicates":rep_vals,
                                                   "Metadata_scheme":scheme_vals,
                                                   "pearson_gt_mahala_cs_perm_reg_l1_norm":pearson_correlations,
                                                    "spearman_gt_mahala_cs_perm_reg_l1_norm":spearman_correlations,
                                                   'fpr':fpr,
                                                   'tp':tp,
                                                   'fp': fp,
                                                   'tn':tn,
                                                   'fn':fn,
                                                  'precision':precision,
                                                   'recall':recall})     
    regression_hit_nomination_stats.to_csv("regression_summary_stats/summary_stats_from_all_reg_sig_coef_permute_"+permute_threshold+"_Dec102021.csv")

CS_run1
2.0_5.0random
2.0_7.0random
4.0_3.0random
4.0_5.0random
4.0_7.0random
8.0_3.0random
8.0_5.0random
8.0_7.0random
CS_run2
2.0_5.0random
2.0_7.0random
4.0_3.0random
4.0_5.0random
4.0_7.0random
8.0_3.0random
8.0_5.0random
8.0_7.0random
CS_run3
2.0_3.0random1
2.0_3.0random2
2.0_7.0random1
2.0_7.0random2
16.0_5.0random1
16.0_5.0random2
32.0_5.0random1
32.0_5.0random2
CS_run1
2.0_5.0random
2.0_7.0random
4.0_3.0random
4.0_5.0random
4.0_7.0random
8.0_3.0random
8.0_5.0random
8.0_7.0random
CS_run2
2.0_5.0random
2.0_7.0random
4.0_3.0random
4.0_5.0random
4.0_7.0random
8.0_3.0random
8.0_5.0random
8.0_7.0random
CS_run3
2.0_3.0random1
2.0_3.0random2
2.0_7.0random1
2.0_7.0random2
16.0_5.0random1
16.0_5.0random2
32.0_5.0random1
32.0_5.0random2
CS_run1
2.0_5.0random
2.0_7.0random
4.0_3.0random
4.0_5.0random
4.0_7.0random
8.0_3.0random
8.0_5.0random
8.0_7.0random
CS_run2
2.0_5.0random
2.0_7.0random
4.0_3.0random
4.0_5.0random
4.0_7.0random
8.0_3.0random
8.0_5.0random
8.0_7.0random
CS_run3
2.0_3.0r